## Import base catalogs

Hatsify DIA object, source and forced source data from PPDB.

In [1]:
import os
import pyarrow as pa
import pyarrow.parquet as pq

from dask.distributed import Client
from hats_import import pipeline_with_client
from hats_import.collection.arguments import ImportArguments
from pathlib import Path

In [2]:
# Path to the RAW parquet data
PPDB_DIR = Path("/sdf/scratch/rubin/ppdb/data/lsstcam")

# Paths to the target OUTPUT directories
TMP_DIR = Path(os.environ["OUTPUT_DIR"]) / "tmp"
TMP_DIR.mkdir(parents=True, exist_ok=True)
print(f"Writing intermediate files to {TMP_DIR}")

Writing intermediate files to /sdf/home/s/stavar/linccf/ppdb/tmp


In [3]:
# Other arguments to test continuous import
START_INDEX = int(os.environ["START_INDEX"])
BATCH_SIZE = int(os.environ["BATCH_SIZE"])
print(f"START_INDEX: {START_INDEX}, BATCH_SIZE: {BATCH_SIZE}")

START_INDEX: 5, BATCH_SIZE: 5


In [4]:
# Initialize Dask client
client = Client(n_workers=16, threads_per_worker=1, local_directory=TMP_DIR)

In [5]:
def get_paths(dataset_type):
    """Return all parquet files for a given dataset type."""
    dataset_name = "".join(word.capitalize() for word in dataset_type.split("_"))
    files = sorted(PPDB_DIR.rglob(f"{dataset_name}.parquet"))
    print(f"Found {len(files)} {dataset_type} parquet files")
    return files


def download_schema(dataset_type, parquet_filepath):
    """Obtain the final schema for a dataset type."""
    with open(parquet_filepath, "rb") as file:
        schema = pq.read_schema(file).remove_metadata()
    schema_table = pa.table(
        {field.name: pa.array([], type=field.type) for field in schema}
    )
    schema_filepath = TMP_DIR / f"{dataset_type}_schema.parquet"
    pq.write_table(schema_table, schema_filepath)
    print(f"Wrote {dataset_type} schema to {schema_filepath}")
    return schema_filepath


def get_paths_and_schema(dataset_type):
    paths = get_paths(dataset_type)
    # Use the first parquet schema as the final schema
    schema_path = download_schema(dataset_type, paths[0])
    return paths, schema_path

### dia_object

In [6]:
paths, schema_path = get_paths_and_schema("dia_object")

Found 175 dia_object parquet files
Wrote dia_object schema to /sdf/home/s/stavar/linccf/ppdb/tmp/dia_object_schema.parquet


In [7]:
args = ImportArguments(
    output_path=TMP_DIR,
    output_artifact_name="dia_object",
    input_file_list=paths[START_INDEX : START_INDEX + BATCH_SIZE],
    file_reader="parquet",
    ra_column="ra",
    dec_column="dec",
    catalog_type="object",
    pixel_threshold=5_000_000,
    # use_schema_file=schema_path,
    simple_progress_bar=True,
    resume=False,
)
pipeline_with_client(args, client)

Finishing : 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.70it/s]


### dia_source

In [8]:
paths, schema_path = get_paths_and_schema("dia_source")

Found 175 dia_source parquet files
Wrote dia_source schema to /sdf/home/s/stavar/linccf/ppdb/tmp/dia_source_schema.parquet


In [9]:
args = ImportArguments(
    output_path=TMP_DIR,
    output_artifact_name="dia_source",
    input_file_list=paths[START_INDEX : START_INDEX + BATCH_SIZE],
    file_reader="parquet",
    ra_column="ra",
    dec_column="dec",
    catalog_type="source",
    pixel_threshold=4_000_000,
    # use_schema_file=schema_path,
    simple_progress_bar=True,
    resume=False,
)
pipeline_with_client(args, client)

Finishing : 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.37it/s]


### dia_forced_source

In [10]:
paths, schema_path = get_paths_and_schema("dia_forced_source")

Found 90 dia_forced_source parquet files
Wrote dia_forced_source schema to /sdf/home/s/stavar/linccf/ppdb/tmp/dia_forced_source_schema.parquet


In [11]:
args = ImportArguments(
    output_path=TMP_DIR,
    output_artifact_name="dia_forced_source",
    input_file_list=paths[START_INDEX : START_INDEX + BATCH_SIZE],
    file_reader="parquet",
    ra_column="ra",
    dec_column="dec",
    catalog_type="source",
    pixel_threshold=25_000_000,
    # use_schema_file=schema_path,
    simple_progress_bar=True,
    resume=False,
)
pipeline_with_client(args, client)

Finishing : 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  8.08it/s]


In [12]:
client.close()